In [1]:
import pandas as pd
import pymysql
from datetime import datetime
from datetime import timedelta
from Investar import Analyzer

class DualMomentum:
    def __init__(self):
        """생성자: KRX 종목코드(codes)를 구하기 위한 MarkgetDB 객체 생성"""
        self.mk = Analyzer.MarketDB()
    
    def get_rltv_momentum(self, start_date, end_date, stock_count):
        """특정 기간 동안 수익률이 제일 높았던 stock_count 개의 종목들 (상대 모멘텀)
            - start_date  : 상대 모멘텀을 구할 시작일자 ('2020-01-01')   
            - end_date    : 상대 모멘텀을 구할 종료일자 ('2020-12-31')
            - stock_count : 상대 모멘텀을 구할 종목수
        """       
        connection = pymysql.connect(host='localhost', port=3306, db='INVESTAR', user='root', passwd='pmugi73', autocommit=True)
        cursor = connection.cursor()
        
        # 사용자가 입력한 시작일자를 DB에서 조회되는 일자로 보정 
        sql = f"select max(date) from daily_price where date <= '{start_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print ("start_date : {} -> returned None".format(sql))
            return
        start_date = result[0].strftime('%Y-%m-%d')


        # 사용자가 입력한 종료일자를 DB에서 조회되는 일자로 보정
        sql = f"select max(date) from daily_price where date <= '{end_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print ("end_date : {} -> returned None".format(sql))
            return
        end_date = result[0].strftime('%Y-%m-%d')


        # KRX 종목별 수익률을 구해서 2차원 리스트 형태로 추가
        rows = []
        columns = ['code', 'company', 'old_price', 'new_price', 'returns']
        for _, code in enumerate(self.mk.codes):            
            sql = f"select close from daily_price "\
                f"where code='{code}' and date='{start_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            old_price = int(result[0])
            sql = f"select close from daily_price "\
                f"where code='{code}' and date='{end_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            new_price = int(result[0])
            returns = (new_price / old_price - 1) * 100
            rows.append([code, self.mk.codes[code], old_price, new_price, 
                returns])


        # 상대 모멘텀 데이터프레임을 생성한 후 수익률순으로 출력
        df = pd.DataFrame(rows, columns=columns)
        df = df[['code', 'company', 'old_price', 'new_price', 'returns']]
        df = df.sort_values(by='returns', ascending=False)
        df = df.head(stock_count)
        df.index = pd.Index(range(stock_count))
        connection.close()
        print(df)
        print(f"\nRelative momentum ({start_date} ~ {end_date}) : "\
            f"{df['returns'].mean():.2f}% \n")
        return df
    
    def get_abs_momentum(self, rltv_momentum, start_date, end_date):
        """특정 기간 동안 상대 모멘텀에 투자했을 때의 평균 수익률 (절대 모멘텀)
            - rltv_momentum : get_rltv_momentum() 함수의 리턴값 (상대 모멘텀)
            - start_date    : 절대 모멘텀을 구할 매수일 ('2020-01-01')   
            - end_date      : 절대 모멘텀을 구할 매도일 ('2020-12-31')
        """
        stockList = list(rltv_momentum['code'])        
        connection = pymysql.connect(host='localhost', port=3306, db='INVESTAR', user='root', passwd='pmugi73', autocommit=True)
        cursor = connection.cursor()


        # 사용자가 입력한 매수일을 DB에서 조회되는 일자로 변경 
        sql = f"select max(date) from daily_price "\
            f"where date <= '{start_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print ("{} -> returned None".format(sql))
            return
        start_date = result[0].strftime('%Y-%m-%d')


        # 사용자가 입력한 매도일을 DB에서 조회되는 일자로 변경 
        sql = f"select max(date) from daily_price "\
            f"where date <= '{end_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print ("{} -> returned None".format(sql))
            return
        end_date = result[0].strftime('%Y-%m-%d')


        # 상대 모멘텀의 종목별 수익률을 구해서 2차원 리스트 형태로 추가
        rows = []
        columns = ['code', 'company', 'old_price', 'new_price', 'returns']
        for _, code in enumerate(stockList):            
            sql = f"select close from daily_price "\
                f"where code='{code}' and date='{start_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            old_price = int(result[0])
            sql = f"select close from daily_price "\
                f"where code='{code}' and date='{end_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            new_price = int(result[0])
            returns = (new_price / old_price - 1) * 100
            rows.append([code, self.mk.codes[code], old_price, new_price,
                returns])


        # 절대 모멘텀 데이터프레임을 생성한 후 수익률순으로 출력
        df = pd.DataFrame(rows, columns=columns)
        df = df[['code', 'company', 'old_price', 'new_price', 'returns']]
        df = df.sort_values(by='returns', ascending=False)
        connection.close()
        print(df)
        print(f"\nAbasolute momentum ({start_date} ~ {end_date}) : "\
            f"{df['returns'].mean():.2f}%")
        return


ModuleNotFoundError: No module named 'Investar'

## 6.7 듀얼 모멘텀 투자
Momentum: 물체를 움직이게 하는 힘.\
주식: 한번 움직이기 시작한 주식 가격이 계속 그 방향으로 나아가려는 성질
### 6.7.1 모멘텀 현상
모멘텀현상: 군집행동, 정박 효과, 확증 편향, 처분 효과 등의 행동 편향 action bias에 의해서 발생\
군집 행동 herding, 정박 효과 anchoring, 확증 편향 confirmation bias, 처분 효과 disposition effect\
#### 모멘텀의 역사
#### 현대의 모멘텀
전년도에 수익률이 가장 높았던 종목들이 이듬해에도 수익률이 높은 경향\
상대강도 relative strength: 상대 강도가 높은 주식이란 26주 이동평균선을 기준으로 더 많이 오른 종목, 상대강도가 높은 주식들을 매수했을 때 26주 이후부터 시장 초과 수익이 발생.\
승자주 매수 및 패자주 매도의 수익률: 효율적 시장 가설에 대한 영향 Returns to Buying winners and Selling Losers: Implication for stock market efficiency
### 6.7.2 듀얼 모멘텀 투자
유진 파마 Eugene F. Fama: 효율적 시장 가설, 모멘텀은 제1의 시장 이례 현상이다.\
**최근 6~12개월 동안의 상대적 수익률이 높은 종목을 매수하는 상대적 모멘텀 전략이 일리가 있어 보이자만, 이미 수익이 난 종목을 매수하기에 위험성이 커진다. 이에 대한 해결책이 바로 절대적 모멘텀 전략으로 상승장에서만 투자하고 하락장에서는 미국 단기 국채나 현금으로 갈아타는 전략이다.**
#### 듀얼 모멘텀 클래스 구조
먼저 상대 모멘텀을 구한 뒤에, 절대 모멘텀을 구한다. 절매 모멘텀을 호출할 때 상대 모멘텀을 인수로 넘겨준다.
### 6.7.3 상대 모멘텀
relative momentum: 특정기간 동안 상대적으로 수익률이 좋았던 n개(10,20,30)을 구한다.
#### 종목별 수익률 계산
상대모멘털: 종목별 수익률, returns = (new_price/old_price-1)*100
#### 상대모멘텀 데이터프레임 생성
### 6.7.4 절대모멘텀
Absolute Momentum: 자산의 가치가 상승하고 있을 때만 투자하고 그렇지 않을 때는 단기 국채를 매수하거나 현금을 보유하는 전략
### 6.7.5 한국형 듀얼 모멘텀 전략
3개월 전략, 한국자산과 해외 자산을 혼합하는 경우에는 12개월 듀얼 모멘텀을 적용\


In [5]:
#import pandas as pd
#import matplotlib.pyplot as plt
#import datetime
#import matplotlib.dates as mdates
#import mplfinance as mpf
#from mplfinance.original_flavor import candlestick_ohlc 
import matplotlib.font_manager as fm

# 폰트 및 설정 (이전에 한글 문제 해결을 위해 추가된 부분)
plt.rcParams['font.family'] = 'Malgun Gothic' 
plt.rcParams['axes.unicode_minus'] = False

import sys
import os

investar_parent_path = 'C:\\myPackage'
if investar_parent_path not in sys.path:
    sys.path.append(investar_parent_path)

from Investar.Analyzer import MarketDB
from Investar.DualMomentum import DualMomentum

mk = MarketDB()
dm = DualMomentum()
rm = dm.get_rltv_momentum('2025-02-26','2025-06-26',20)
#am = dm.get_abs_momentum(rm,'2025-05-02','2025-11-26')
am = dm.get_abs_momentum(rm,'2025-06-26','2025-11-26')

      code  company  old_price  new_price      returns
0   012170     아센디오        230       3190  1286.956522
1   036630    세종텔레콤        424       5020  1083.962264
2   258050    테크트랜스        266       2200   727.067669
3   405000     플라즈맵       1040       8590   725.961538
4   352770    셀레스트라        695       5250   655.395683
5   214610     롤링스톤        805       5450   577.018634
6   042510    라온시큐어       2140      12030   462.149533
7   065060      지엔코        252       1310   419.841270
8   222810     세토피아       1080       5400   400.000000
9   307280    원바이오젠       1138       5410   375.395431
10  131090      시큐브        904       4280   373.451327
11  009310   참엔지니어링        351       1423   305.413105
12  208710       포톤        630       2550   304.761905
13  124500  아이티센글로벌       6190      24400   294.184168
14  060240    스타코링크        535       1976   269.345794
15  088800    에이스테크        759       2750   262.318841
16  377030     비트맥스       2140       7120   232.710280
17  419530